In [1]:
import sys
sys.path.append('../')
import numpy as np
from GroupingAlgorithm import *
from HEEM_VQE_Functions import *
from utils import Label2Chain, get_backend_conectivity
from qiskit import IBMQ
from qiskit_nature.drivers import PyQuanteDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit.opflow import converters
from qiskit.opflow.primitive_ops import Z2Symmetries

Obtaing the conectivity of a real quantum hardware.

In [2]:
IBMQ.load_account()
provider    = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') # put here your provider
backend_exp = provider.get_backend('ibmq_paris') # select a quantum computer
WC_exp      = get_backend_conectivity(backend_exp)

Groups for different molecules. The grouping strategies considered are: non grouping, TPB, Bell+TPB, 2Q+TPB and HEEM.

In [3]:
molecules = [ 'H .0 .0 .0; H .0 .0 0.761'  #0.88
             ,
             'Li 0.0 0.0 0.0; H 0.0 0.0 1.619'
             ,
            'H 0.0 0.0 -1.339; Be 0.0 0.0 0.0; H 0.0 0.0 1.339'
             ,
            'O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0'
#              ,
#              'H 0.0 0.0 0.0; F 0.0 0.0 0.995'
#              ,
#             'Cl 0.0 0.0 0.0; H 0.0 0.0 1.0' 
            ]


for molecule in molecules :
    
    print( 'molecule: '+molecule )

    driver = PyQuanteDriver(molecule)
    qmolecule = driver.run()
    problem = ElectronicStructureProblem(driver)

    # Generate the second-quantized operators
    second_q_ops = problem.second_q_ops()

    # Hamiltonian
    main_op = second_q_ops[0]

    # Setup the mapper and qubit converter
    mapper = JordanWignerMapper()

    converter = QubitConverter(mapper=mapper)

    # The fermionic operators are mapped to qubit operators
    num_particles = (problem.molecule_data_transformed.num_alpha,
                 problem.molecule_data_transformed.num_beta)
    qubit_op   = converter.convert(main_op, num_particles=num_particles)

    num_qubits = qubit_op.num_qubits
    WC=list(range(num_qubits))
    WC=list(permutations(list(range(num_qubits)),2))

    print('number of qubits:', num_qubits)

    paulis, coeff, labels = Label2Chain(qubit_op)
    print( 'number of Paulis:', len(coeff) )
    
    Color, Groups_tpb, Measurements_tpb = TPBgrouping(paulis)
    print( 'number of TPB groups:', len(Groups_tpb) )
        
    Groups_bell, Measurements_bell = grouping( paulis,[4,3,1,2],WC)
    print('number of Bell groups:', len(Groups_bell) )
    
    Groups_2Q1, Measurements_2Q1 = grouping( paulis,[4,3,2,1,9,8,7,6,5],WC)
    print('number of 2Q groups v1:', len(Groups_2Q1) )

    Groups_2Q2, Measurements_2Q2 = grouping( paulis,[4,3,2,1,7,6,5,9,8],WC)
    print('number of 2Q groups v2:', len(Groups_2Q2) )
    
    Groups_HEEM, Measurements_HEEM = grouping( paulis,[4,3,1,2],WC_exp)
    print('number of HEEM groups :', len(Groups_HEEM) )
    
    print('----------------------------------------')


molecule: H .0 .0 .0; H .0 .0 0.761
number of qubits: 4
number of Paulis: 15
number of TPB groups: 5
number of Bell groups: 2
number of 2Q groups v1: 2
number of 2Q groups v2: 2
number of HEEM groups : 2
----------------------------------------
molecule: Li 0.0 0.0 0.0; H 0.0 0.0 1.619
number of qubits: 12
number of Paulis: 631
number of TPB groups: 136
number of Bell groups: 39
number of 2Q groups v1: 39
number of 2Q groups v2: 39
number of HEEM groups : 101
----------------------------------------
molecule: H 0.0 0.0 -1.339; Be 0.0 0.0 0.0; H 0.0 0.0 1.339
number of qubits: 14
number of Paulis: 666
number of TPB groups: 140
number of Bell groups: 41
number of 2Q groups v1: 41
number of 2Q groups v2: 41
number of HEEM groups : 126
----------------------------------------
molecule: O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0
number of qubits: 14
number of Paulis: 1086
number of TPB groups: 224
number of Bell groups: 58
number of 2Q groups v1: 58
number of 2Q groups v2: 58
numb

In [4]:
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.619'

driver = PyQuanteDriver(molecule)
qmolecule = driver.run()

freezeCoreTransfomer = FreezeCoreTransformer(freeze_core=True, remove_orbitals=[3, 4])
problem = ElectronicStructureProblem(driver, q_molecule_transformers=[freezeCoreTransfomer])

# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]

# Setup the mapper and qubit converter
mapper = ParityMapper()

converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)

# The fermionic operators are mapped to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
qubit_op   = converter.convert(main_op, num_particles=num_particles)

num_qubits = qubit_op.num_qubits
WC=list(range(num_qubits))
WC=list(permutations(list(range(num_qubits)),2))

print('number of qubits:', num_qubits)

paulis, coeff, labels = Label2Chain(qubit_op)
print( 'number of Paulis:', len(coeff) )

Color, Groups_tpb, Measurements_tpb = TPBgrouping(paulis)
print( 'number of TPB groups:', len(Groups_tpb) )

Groups_bell, Measurements_bell = grouping( paulis,[4,3,1,2],WC)
print('number of Bell groups:', len(Groups_bell) )

Groups_2Q1, Measurements_2Q1 = grouping( paulis,[4,9,8,7,6,5,3,2,1],WC)
print('number of 2Q groups v1:', len(Groups_2Q1) )

Groups_2Q2, Measurements_2Q2 = grouping( paulis,[7,6,5,4,9,8,3,2,1],WC)
print('number of 2Q groups v2:', len(Groups_2Q1) )

Groups_HEEM, Measurements_HEEM = grouping( paulis,[4,3,2,1],WC_exp)
print('number of HEEM groups :', len(Measurements_HEEM) )

print('----------------------------------------')


number of qubits: 4
number of Paulis: 100
number of TPB groups: 25
number of Bell groups: 15
number of 2Q groups v1: 11
number of 2Q groups v2: 11
number of HEEM groups : 9
----------------------------------------
